In [176]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import math
import os
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset
from torchvision.transforms import ToTensor
from pathlib import Path
import requests
import torch.nn.functional as F

from torch.utils.tensorboard import SummaryWriter

In [124]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [185]:
#Getting data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

writer = SummaryWriter("runs/mnist2")

class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)
    
batch_size = 100;

#defining dataset and dataloader
#train_ds = TensorDataset(training_data.data.reshape(-1,28*28).float(), training_data.targets)
#valid_ds = TensorDataset(test_data.data.reshape(-1,28*28).float(), test_data.targets)

def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )

class Mnist_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)
        
    def forward(self, xb):
        xb = xb.view(-1,1,28,28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return F.log_softmax(xb.view(-1, xb.size(1)), dim=1)

#defining loss function, weights, and bias tensors
loss_func = F.nll_loss
lr = 0.05

def get_model():
    model = Mnist_Logistic()
    return model, optim.SGD(model.parameters(),lr=lr)


def accuracy(xb, yb):
    max_xb = torch.argmax(xb,dim=1)
    return (max_xb == yb).float().mean()

def loss_batch(model, loss_func, xb, yb, opt = None):
    loss = loss_func(model(xb),yb)
    
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    return loss.item(), len(xb)

def acc_batch(model, xb, yb):
    acc = accuracy(model(xb),yb)
    return acc, len(xb)
        
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    n_total_steps = len(train_dl)
    running_loss = 0.0
    running_acc = 0

    for epoch in range(epochs):
        i=0
        model.train()
        for xb,yb in train_dl:
            i+=1
            running_loss += loss_batch(model,loss_func,xb, yb, opt)[0]
            if i%100==0:
                writer.add_scalar('training loss',running_loss/100, epoch*n_total_steps + i)
                running_loss = 0
        model.eval()
        with torch.no_grad():
            losses, nums = zip(*[loss_batch(model, loss_func, xb, yb) for xb,yb in valid_dl])
            accs, nums = zip(*[acc_batch(model, xb, yb) for xb,yb in valid_dl])
        val_loss = np.sum(np.multiply(losses,nums))/np.sum(nums)
        val_acc = np.sum(np.multiply(accs,nums))/np.sum(nums)
        print(epoch, val_loss,val_acc)

        

In [151]:
train_dl, valid_dl = get_data(training_data,test_data,batch_size)
examples = iter(train_dl)
example_data, example_targets = next(examples)
img_grid = torchvision.utils.make_grid(example_data)
writer.add_image('mnist_images_5',img_grid)
writer.close()

In [149]:
example_data.shape

torch.Size([100, 1, 28, 28])

In [186]:
total_epochs = 5;
train_dl, valid_dl = get_data(training_data,test_data,batch_size)
model = Mnist_CNN()
opt = optim.SGD(model.parameters(), lr = lr, momentum = 0.9)
writer.add_graph(model, example_data)
writer.close()
fit(total_epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.7431335246562958 0.7125999963283539
1 0.5921678930521012 0.7962999987602234
2 0.557761258482933 0.8027000010013581
3 0.49311653077602385 0.820200001001358
4 0.4716861736774445 0.828600001335144


In [175]:
model(example_data)[0:3].exp()

tensor([[5.5868e-03, 9.5361e-01, 4.3575e-05, 3.9910e-02, 2.9719e-04, 1.1410e-05,
         3.9765e-04, 6.9662e-05, 6.9238e-05, 3.8486e-06],
        [6.3568e-05, 9.5347e-06, 1.4743e-05, 7.2948e-05, 5.0899e-06, 7.5699e-02,
         4.8607e-05, 9.2089e-01, 1.7275e-03, 1.4704e-03],
        [2.2801e-04, 7.3300e-06, 9.4397e-05, 2.2896e-03, 1.2300e-04, 4.8264e-04,
         6.0899e-05, 9.4662e-04, 3.8296e-04, 9.9538e-01]],
       grad_fn=<ExpBackward0>)

In [173]:
example_targets[0:3]

tensor([1, 7, 9])

In [159]:
model(example_data)[0].sum()

tensor(-74.0803, grad_fn=<SumBackward0>)

In [163]:
example_data.view(-1,1,28,28).shape

torch.Size([100, 1, 28, 28])